In [1]:
!git clone https://github.com/recommenders-team/recommenders.git


Cloning into 'recommenders'...
remote: Enumerating objects: 43247, done.
remote: Counting objects: 100% (1718/1718), done.
remote: Compressing objects: 100% (673/673), done.
remote: Total 43247 (delta 1298), reused 1072 (delta 1044), pack-reused 41529 (from 4)
Receiving objects: 100% (43247/43247), 223.23 MiB | 32.51 MiB/s, done.
Resolving deltas: 100% (28934/28934), done.


In [2]:
%pip install retrying

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip uninstall -y tensorflow
!pip install tensorflow==2.15.0

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 94.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.4.

In [4]:
import os
import sys
import numpy as np
import zipfile
from tqdm import tqdm
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages
import sys

sys.path.append('/kaggle/working/recommenders/')

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.naml import NAMLModel
from recommenders.models.newsrec.io.mind_all_iterator import MINDAllIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set
from recommenders.utils.notebook_utils import store_metadata

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]
Tensorflow version: 2.15.0


In [5]:
epochs = 5
seed = 42
batch_size = 32

# /kaggle/input/test-naml-amz/processed_data

In [6]:
train_news_file = "/kaggle/input/amazon-3/processed_data/train/news.tsv"
train_behaviors_file = "/kaggle/input/amazon-3/processed_data/train/behaviors.tsv"
valid_news_file = "/kaggle/input/amazon-3/processed_data/valid/news.tsv"
valid_behaviors_file = "/kaggle/input/amazon-3/processed_data/valid/behaviors_more_neg.tsv"
wordEmb_file = "/kaggle/input/amazon-3/processed_data/utils/embedding_all_300d.npy"
userDict_file = "/kaggle/input/amazon-3/processed_data/utils/uid2index.pkl"
wordDict_file = "/kaggle/input/amazon-3/processed_data/utils/word_dict_all.pkl"
vertDict_file = "/kaggle/input/amazon-3/processed_data/utils/vert_dict.pkl"
subvertDict_file = "/kaggle/input/amazon-3/processed_data/utils/subvert_dict.pkl"
yaml_file = "/kaggle/input/amazon-3/processed_data/naml.yaml"

In [7]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          vertDict_file=vertDict_file, 
                          subvertDict_file=subvertDict_file,
                          batch_size=batch_size,
                          epochs=epochs)
print(hparams)

HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 4, 'head_dim': 100, 'filter_num': 400, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 5, 'batch_size': 32, 'show_step': 100000, 'title_size': 30, 'body_size': 50, 'his_size': 50, 'vert_num': 2, 'subvert_num': 673, 'data_format': 'naml', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10', 'hit@5;10;20'], 'word_emb_dim': 300, 'cnn_activation': 'relu', 'model_type': 'naml', 'dense_activation': 'relu', 'loss': 'cross_entropy_loss', 'wordEmb_file': '/kaggle/input/amazon-3/processed_data/utils/embedding_all_300d.npy', 'wordDict_file': '/kaggle/input/amazon-3/processed_data/utils/word_dict_all.pkl', 'userDict_file': '/kaggle/input/amazon-3/processed_data/utils/uid2index.pkl', 'vertDict_file': '/kaggle/input/amazon-3/processed_data/utils/vert_dic

In [8]:
iterator = MINDAllIterator

In [9]:
model = NAMLModel(hparams, iterator, seed=seed)

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [10]:
%%time
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

312it [01:46,  2.92it/s]
0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
152392it [05:11, 489.22it/s]
9969it [00:28, 351.01it/s]
10000it [00:00, 18262.61it/s]


at epoch 1
train info: logloss loss:1.576627912429663
eval info: group_auc:0.6038, hit@10:0.6245, hit@20:1.0, hit@5:0.3897, mean_mrr:0.2838, ndcg@10:0.3427, ndcg@5:0.2673
at epoch 1 , train time: 107.0 eval time: 351.3


312it [01:32,  3.38it/s]
152392it [05:03, 502.19it/s]
9969it [00:27, 366.16it/s]
10000it [00:00, 18302.70it/s]


at epoch 2
train info: logloss loss:1.4335906467376611
eval info: group_auc:0.6399, hit@10:0.6769, hit@20:1.0, hit@5:0.4437, mean_mrr:0.3253, ndcg@10:0.3896, ndcg@5:0.315
at epoch 2 , train time: 92.2 eval time: 342.5


312it [01:32,  3.38it/s]
152392it [05:03, 502.54it/s]
9969it [00:27, 366.02it/s]
10000it [00:00, 18509.64it/s]


at epoch 3
train info: logloss loss:1.3356296381889246
eval info: group_auc:0.6496, hit@10:0.6823, hit@20:1.0, hit@5:0.4559, mean_mrr:0.342, ndcg@10:0.404, ndcg@5:0.3314
at epoch 3 , train time: 92.3 eval time: 341.9


312it [01:32,  3.38it/s]
152392it [05:02, 504.03it/s]
9969it [00:27, 367.95it/s]
10000it [00:00, 18497.88it/s]


at epoch 4
train info: logloss loss:1.26444366421455
eval info: group_auc:0.6566, hit@10:0.688, hit@20:1.0, hit@5:0.4672, mean_mrr:0.3478, ndcg@10:0.4101, ndcg@5:0.3394
at epoch 4 , train time: 92.4 eval time: 340.8


312it [01:32,  3.38it/s]
152392it [05:02, 504.18it/s]
9969it [00:27, 369.17it/s]
10000it [00:00, 18701.19it/s]


at epoch 5
train info: logloss loss:1.186700711074548
eval info: group_auc:0.6613, hit@10:0.6905, hit@20:1.0, hit@5:0.4784, mean_mrr:0.3549, ndcg@10:0.4165, ndcg@5:0.3485
at epoch 5 , train time: 92.4 eval time: 340.7
CPU times: user 54min, sys: 6min 52s, total: 1h 53s
Wall time: 36min 33s


In [11]:
test_news_file = "/kaggle/input/amazon-3/processed_data/test/news.tsv"
test_behaviors_file = "/kaggle/input/amazon-3/processed_data/test/behaviors_more_neg.tsv"

In [12]:
%%time
res_syn = model.run_eval(test_news_file, test_behaviors_file)
print(res_syn)

152392it [05:02, 504.52it/s]
9969it [00:26, 371.25it/s]
10000it [00:00, 18552.37it/s]


{'group_auc': 0.6613, 'mean_mrr': 0.3549, 'ndcg@5': 0.3485, 'ndcg@10': 0.4165, 'hit@5': 0.4784, 'hit@10': 0.6905, 'hit@20': 1.0}
CPU times: user 9min 45s, sys: 1min 17s, total: 11min 3s
Wall time: 5min 40s


In [13]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)

152392it [05:02, 503.32it/s]
9969it [00:27, 367.03it/s]
10000it [00:00, 18197.45it/s]


{'group_auc': 0.6613, 'mean_mrr': 0.3549, 'ndcg@5': 0.3485, 'ndcg@10': 0.4165, 'hit@5': 0.4784, 'hit@10': 0.6905, 'hit@20': 1.0}
CPU times: user 9min 48s, sys: 1min 19s, total: 11min 8s
Wall time: 5min 41s


In [14]:
# # Record results for tests - ignore this cell
# store_metadata("group_auc", res_syn['group_auc'])
# store_metadata("mean_mrr", res_syn['mean_mrr'])
# store_metadata("ndcg@5", res_syn['ndcg@5'])
# store_metadata("ndcg@10", res_syn['ndcg@10'])

In [15]:
model_path = "/kaggle/working/model"
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "naml_ckpt"))
